# Pattern Mining - Association Rule Mining

A *frequent pattern* is a substructure that appears frequently in a dataset. Finding the frequent patterns of a dataset is a essential step in data mining tasks such as *feature extraction* and a necessary ingredient of *association rule learning*. This kind of algorithms are extremely useful in the field of *Market Basket Analysis*, which in turn provide retailers with invaluable information about their customer shopping habbits and needs. 

Here, I will shortly describe the *GLC frequent pattern mining toolkit*, the tools it provides and its functionality. Major advantage of this high-level ML toolkit is the ease it provides to train an association rule mining algorithm, as well as the high interpretability of the returned results. Under the hood the GLG frequent pattern mining toolkit runs a *[TFP-Growth algorithm](https://www.computer.org/csdl/trans/tk/2005/05/k0652-abs.html)*, introduced by Wang, Jianyong, et al. in 2005. For a recent review of the various directions in the field consult *[Han, Jiawei, et al. "Frequent pattern mining: current status and future directions.", Data Mining and Knowledge Discovery 15.1 (2007): 55-86](http://link.springer.com/article/10.1007%2Fs10618-006-0059-1)*.

## Fire up GraphLab Create

In [1]:
import graphlab as gl
from graphlab import aggregate as agg

## A simple retailer example

Here we discuss a simple example of receipt data from a bakery. The dataset consists of items like `'ApplePie'` and `'GanacheCookie'`. The task is to identify sets of items that are frequently bought together. The dataset consists of *266209 rows* and *6 columns* which look like the following. The dataset was constructed by modifying the *[Extended BAKERY dataset](https://wiki.csc.calpoly.edu/datasets/wiki/ExtendedBakery)*.

In [2]:
bakery_sf = gl.SFrame('./bakery_sf')
bakery_sf

2016-05-09 01:13:58,787 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1462745636.log


This non-commercial license of GraphLab Create is assigned to tgrammat@gmail.com and will expire on September 21, 2016. For commercial licensing options, visit https://dato.com/buy/.


Receipt,SaleDate,EmpId,StoreNum,Quantity,Item
1,12-JAN-2000,20,20,1,GanacheCookie
1,12-JAN-2000,20,20,5,ApplePie
2,15-JAN-2000,35,10,1,CoffeeEclair
2,15-JAN-2000,35,10,3,ApplePie
2,15-JAN-2000,35,10,4,AlmondTwist
2,15-JAN-2000,35,10,3,HotCoffee
3,8-JAN-2000,13,13,5,OperaCake
3,8-JAN-2000,13,13,3,OrangeJuice
3,8-JAN-2000,13,13,3,CheeseCroissant
4,24-JAN-2000,16,16,1,TruffleCake


As we can see below, all the coffee products have similar sale frequencies and there is no some particular subset of products that is more preferred than the remaining ones.

In [3]:
gl.canvas.set_target('ipynb')
bakery_sf['Item'].show()

Next, we split the **`bakery_sf`** data set in a **training** and a **test part**.

In [4]:
(train, test) = bakery_sf.random_split(0.8, seed=1)

In [5]:
print 'Number of Rows in training set [80pct of Known Examples]: %d'   % train.num_rows()
print 'Number of Rows in test set [20pct of Known Examples]: %d'   % test.num_rows()

Number of Rows in training set [80pct of Known Examples]: 212930
Number of Rows in test set [20pct of Known Examples]: 53279


In order to run a frequent pattern mining algorithm, we require an **item column**, (the column **`'Item'`** in this example), and a set of **feature columns** that *uniquely identify* a transaction (the columns **`['Receipt', 'StoreNum']`** in this example, since we need to take in account the geophraphical location of each store and the accompanied social-economic criteria that may exist).

In addition we need to specify the 3 basic parameters of the **FP-Growth algorithm** which is called by the high-level GraphLab Create (GLC) function. These are:

* **`min_support`:** The minimum number of times that a pattern must occur in order to be considered a *frequent* one. Here, we choose a threshold of 1&permil; of total transactions in record to be the `min_support`.
* **`max_patterns`:** The maximum number of frequent patterns to be mined.
* **`min_length`:** The minimum size (number of elements in the set) of each pattern being mined.

In [6]:
min_support = int(train.num_rows()*0.001)
model = gl.frequent_pattern_mining.create(train, 'Item', features=['Receipt', 'StoreNum'], 
                                          min_support=min_support, max_patterns=500, min_length=4)

Indexing complete. Found 50 unique items.

Preprocessing complete. Found 73566 unique transactions.

Building frequent pattern tree.

+-----------+----------------+------------+----------------------+------------------+

| Iteration | Num. Patterns  | Support    | Current Min Support  | Elapsed Time     |

...

| 48        | 8              | 2521       | 212                  | 71.781ms         |

| 49        | 8              | 2511       | 212                  | 77.793ms         |

| Final     | 8              | -          | 212                  | 79.672ms         |

+-----------+----------------+------------+----------------------+------------------+

Pattern mining complete. Found 8 unique closed patterns.

Here, we obtain the most frequent feature patterns.

In [7]:
print 'The most frequent feature patters are:'
print '-----------------------------------------'
model.frequent_patterns.print_rows(max_column_width=70)

The most frequent feature patters are:
-----------------------------------------
+-----------------------------------------------------------------------+---------+
|                                pattern                                | support |
+-----------------------------------------------------------------------+---------+
|            [CoffeeEclair, HotCoffee, ApplePie, AlmondTwist]           |   877   |
|    [LemonLemonade, RaspberryCookie, LemonCookie, RaspberryLemonade]   |   813   |
|        [LemonLemonade, RaspberryCookie, LemonCookie, GreenTea]        |   649   |
|      [RaspberryCookie, LemonCookie, RaspberryLemonade, GreenTea]      |   645   |
|          [AppleTart, AppleDanish, AppleCroissant, CherrySoda]         |   640   |
|       [LemonLemonade, LemonCookie, RaspberryLemonade, GreenTea]       |   630   |
|     [LemonLemonade, RaspberryCookie, RaspberryLemonade, GreenTea]     |   620   |
| [LemonLemonade, RaspberryCookie, LemonCookie, RaspberryLemonade, G... |   512

Note that the **`'pattern'`** column contains the patterns that occur frequently together, whereas the **`'support'`** column contains the number of times these patterns occur together in the entire dataset. 

In this example, the pattern:
```
[CoffeeEclair, HotCoffee, ApplePie, AlmondTwist]
```
occurred 877 times in the training data.


**Definition**

> A **frequent pattern** is a *set of items* with a *support greater than* user-specified **minimum support threshold**.

However, there is significant redundancy in mining frequent patterns; every subset of a frequent pattern is also frequent (e.g. `'CoffeeEclair'` must be frequent if `['CoffeeEclair'`, `'HotCoffee']` is frequent). The frequent pattern mining toolkit avoids this redundancy by mining the closed frequent patterns, i.e. *frequent patterns with no superset of the same support*. This is achieved by the very design of the **FP-Growth Algorithm**.

Note, that by relaxing the **`min_length`** requirement, one can obtain more frequent patterns of sold coffe products.

In [8]:
min_support = int(train.num_rows()*0.001)
model = gl.frequent_pattern_mining.create(train, 'Item', features=['Receipt', 'StoreNum'], 
                                          min_support=min_support, max_patterns=500, min_length=3)

Indexing complete. Found 50 unique items.

Preprocessing complete. Found 73566 unique transactions.

Building frequent pattern tree.

+-----------+----------------+------------+----------------------+------------------+

| Iteration | Num. Patterns  | Support    | Current Min Support  | Elapsed Time     |

...

| 48        | 31             | 2521       | 212                  | 71.269ms         |

| 49        | 31             | 2511       | 212                  | 72.901ms         |

| Final     | 31             | -          | 212                  | 77.761ms         |

+-----------+----------------+------------+----------------------+------------------+

Pattern mining complete. Found 31 unique closed patterns.

In [9]:
print 'The most frequent feature patters are:'
print '-----------------------------------------'
model.frequent_patterns.print_rows(num_rows=35, max_column_width=70)

The most frequent feature patters are:
-----------------------------------------
+-----------------------------------------------------------------------+---------+
|                                pattern                                | support |
+-----------------------------------------------------------------------+---------+
|                 [CherryTart, ApricotDanish, OperaCake]                |   1561  |
|                 [CoffeeEclair, ApplePie, AlmondTwist]                 |   1351  |
|              [ChocolateCake, ChocolateCoffee, CasinoCake]             |   1308  |
|              [HotCoffee, ApricotCroissant, BlueberryTart]             |   1220  |
|                  [CoffeeEclair, HotCoffee, ApplePie]                  |   1109  |
|                   [HotCoffee, ApplePie, AlmondTwist]                  |   1100  |
|                 [CoffeeEclair, HotCoffee, AlmondTwist]                |   1086  |
|             [CoffeeEclair, BlackberryTart, SingleEspresso]            |   102

Alternatively, by decreasing the **`min_support`** one can obtain more patterns of sold coffee products which are again assumed frequent but with respect to this new threshold.

In [10]:
min_support = int(train.num_rows()*(1e-04))
model = gl.frequent_pattern_mining.create(train, 'Item', features=['Receipt', 'StoreNum'], 
                                          min_support=min_support, max_patterns=500, min_length=4)

Indexing complete. Found 50 unique items.

Preprocessing complete. Found 73566 unique transactions.

Building frequent pattern tree.

+-----------+----------------+------------+----------------------+------------------+

| Iteration | Num. Patterns  | Support    | Current Min Support  | Elapsed Time     |

...

| 48        | 57             | 2521       | 21                   | 234.381ms        |

| 49        | 58             | 2511       | 21                   | 244.784ms        |

| Final     | 59             | -          | 21                   | 255.337ms        |

+-----------+----------------+------------+----------------------+------------------+

Pattern mining complete. Found 59 unique closed patterns.

In [11]:
print 'The most frequent feature patters are:'
print '-----------------------------------------'
model.frequent_patterns.print_rows(num_rows=60, max_column_width=70)

The most frequent feature patters are:
-----------------------------------------
+-----------------------------------------------------------------------+---------+
|                                pattern                                | support |
+-----------------------------------------------------------------------+---------+
|            [CoffeeEclair, HotCoffee, ApplePie, AlmondTwist]           |   877   |
|    [LemonLemonade, RaspberryCookie, LemonCookie, RaspberryLemonade]   |   813   |
|        [LemonLemonade, RaspberryCookie, LemonCookie, GreenTea]        |   649   |
|      [RaspberryCookie, LemonCookie, RaspberryLemonade, GreenTea]      |   645   |
|          [AppleTart, AppleDanish, AppleCroissant, CherrySoda]         |   640   |
|       [LemonLemonade, LemonCookie, RaspberryLemonade, GreenTea]       |   630   |
|     [LemonLemonade, RaspberryCookie, RaspberryLemonade, GreenTea]     |   620   |
| [LemonLemonade, RaspberryCookie, LemonCookie, RaspberryLemonade, G... |   512

To see some details of the trained model:

In [12]:
print model

Class                         : FrequentPatternMiner

Model fields
------------
Min support                   : 21
Max patterns                  : 500
Min pattern length            : 4

Most frequent patterns
----------------------
['CoffeeEclair', 'HotCoffee', 'ApplePie', 'AlmondTwist']: 877
['LemonLemonade', 'RaspberryCookie', 'LemonCookie', 'RaspberryLemonade']: 813
['LemonLemonade', 'RaspberryCookie', 'LemonCookie', 'GreenTea']: 649
['RaspberryCookie', 'LemonCookie', 'RaspberryLemonade', 'GreenTea']: 645
['AppleTart', 'AppleDanish', 'AppleCroissant', 'CherrySoda']: 640
['LemonLemonade', 'LemonCookie', 'RaspberryLemonade', 'GreenTea']: 630
['LemonLemonade', 'RaspberryCookie', 'RaspberryLemonade', 'GreenTea']: 620
['LemonLemonade', 'RaspberryCookie', 'LemonCookie', 'RaspberryLemonade', 'GreenTea']: 512
['HotCoffee', 'CherryTart', 'ApricotDanish', 'OperaCake']: 29
['CherryTart', 'ApricotDanish', 'OperaCake', 'AlmondBear Claw']: 29



## Top-k frequent patterns

In practice, we rarely know the appropriate **`min_support`** threshold to use. As an alternative to specifying a minimum support, we can specify a **maximum number of patterns** to mine using the **`max_patterns`** parameter. Instead of mining all patterns above a minimum support threshold, we mine the most frequent patterns until the maximum number of closed patterns are round. For large data sets, this mining process can be time-consuming. We recommend specifying a somehow *large initial minimum support* bound to speed up the mining.

In [13]:
min_support = int(train.num_rows()*1e-03)
top5_freq_patterns = gl.frequent_pattern_mining.create(train, 'Item',
                                                       features=['Receipt', 'StoreNum'],
                                                       min_support=min_support,
                                                       max_patterns=5,
                                                       min_length=4)

Indexing complete. Found 50 unique items.

Preprocessing complete. Found 73566 unique transactions.

Building frequent pattern tree.

+-----------+----------------+------------+----------------------+------------------+

| Iteration | Num. Patterns  | Support    | Current Min Support  | Elapsed Time     |

...

| 48        | 5              | 2521       | 640                  | 42.609ms         |

| 49        | 5              | 2511       | 640                  | 44.434ms         |

| Final     | 5              | -          | 640                  | 46.272ms         |

+-----------+----------------+------------+----------------------+------------------+

Pattern mining complete. Found 5 unique closed patterns.

In [14]:
print top5_freq_patterns

Class                         : FrequentPatternMiner

Model fields
------------
Min support                   : 212
Max patterns                  : 5
Min pattern length            : 4

Most frequent patterns
----------------------
['CoffeeEclair', 'HotCoffee', 'ApplePie', 'AlmondTwist']: 877
['LemonLemonade', 'RaspberryCookie', 'LemonCookie', 'RaspberryLemonade']: 813
['LemonLemonade', 'RaspberryCookie', 'LemonCookie', 'GreenTea']: 649
['RaspberryCookie', 'LemonCookie', 'RaspberryLemonade', 'GreenTea']: 645
['AppleTart', 'AppleDanish', 'AppleCroissant', 'CherrySoda']: 640



We can always save the trained model by calling:

In [15]:
top5_freq_patterns.save('top5_freq_patterns_model')

### Business Application: Compute Association Rules and Make Predictions

An association rule is an ordered pair of item sets (prefix $A$, prediction $B$) denoted $A\Rightarrow B$ such that A and B are disjoint whereas $A\cup B$ is frequent. The most popular criteria for scoring association rules is to measure the **confidence of the rule**: *the ratio of the support of $A\cup B$ to the support of $A$*.

$$
Confidence(A\Rightarrow B) = \fraq{\textrm{Supp}(A\cup B)}{\textrm{Supp}(A)}.
$$

The confidence of the rule $A\Rightarrow B$ is our empirical estimate of the conditional probability for $B$ given $A$.

One can make predictions using the **`predict()`** or **`predict_topk()`** method for single and multiple predictions respectively. The output of both the methods is an **SFrame** with the following columns:

* **prefix**: The antecedent or left-hand side of an association rule. It must be a frequent pattern and a subset of the associated pattern.
* **prediction**: The consequent or right-hand side of the association rule. It must be disjoint of the prefix.
* **confidence**: The confidence of the association rule as defined above.
* **prefix support**: The frequency of the prefix pattern in the training data.
* **joint support**: The frequency of the co-occurrence ( prefix + prediction) in the training data

In [16]:
predictions = top5_freq_patterns.predict(test)

Preprocessing complete. Found 39293 unique transactions.

In [17]:
predictions.print_rows(max_column_width=100)

+---------+----------+----------------------------+--------------------+-----------------+----------------+
| Receipt | StoreNum |           prefix           |     prediction     |    confidence   | prefix support |
+---------+----------+----------------------------+--------------------+-----------------+----------------+
|  35020  |    14    |             []             |   [CoffeeEclair]   | 0.0890356958378 |     73566      |
|   6447  |    5     | [BlueberryTart, HotCoffee] | [ApricotCroissant] |  0.728793309438 |      1674      |
|  12900  |    10    |      [BlueberryTart]       | [ApricotCroissant] |  0.412691996766 |      4948      |
|  49658  |    20    |      [BlueberryTart]       | [ApricotCroissant] |  0.412691996766 |      4948      |
|  20227  |    19    |    [RaspberryLemonade]     | [RaspberryCookie]  |  0.334982681841 |      4042      |
|  30891  |    12    |      [BlueberryTart]       | [ApricotCroissant] |  0.412691996766 |      4948      |
|  47850  |    2     |      

## Feature Engineering to help other ML Tasks in Pipeline 

### The `.extract_feature()` method

Using the set of closed patterns, we can convert pattern data to binary features vectors. These feature vectors can be used for other machine learning tasks, such as **clustering** or **classification**. For each input pattern `x`, the `j-th` extracted feature `f_{x}[j]` is a binary indicator of whether the `j-th` closed pattern is contained in `x`.

First, we train the **`top100_freq_patterns`** model as shown below:

In [18]:
top100_freq_patterns = gl.frequent_pattern_mining.create(train, 'Item',
                                                         features=['Receipt', 'StoreNum'],
                                                         min_support=1, # occurs at least once in our data record
                                                         max_patterns = 100, # do not search for more than 100 patterns
                                                         min_length=2) # test data have only one coffee product sold per tid . We search for patterns of at least 2 coffee products

Indexing complete. Found 50 unique items.

Preprocessing complete. Found 73566 unique transactions.

Building frequent pattern tree.

+-----------+----------------+------------+----------------------+------------------+

| Iteration | Num. Patterns  | Support    | Current Min Support  | Elapsed Time     |

...

| 48        | 100            | 2521       | 259                  | 101.47ms         |

| 49        | 100            | 2511       | 259                  | 104.547ms        |

| Final     | 100            | -          | 259                  | 107.66ms         |

+-----------+----------------+------------+----------------------+------------------+

Pattern mining complete. Found 100 unique closed patterns.

Here are the 100 unique closed patterns which are found frequent:

In [19]:
top100_freq_patterns.frequent_patterns.print_rows(num_rows=100, max_column_width=50)

+---------------------------------------------------+---------+
|                      pattern                      | support |
+---------------------------------------------------+---------+
|            [CherryTart, ApricotDanish]            |   2542  |
|           [TuileCookie, MarzipanCookie]           |   2419  |
|          [ChocolateCake, ChocolateCoffee]         |   2176  |
|           [GongolaisCookie, TruffleCake]          |   2131  |
|           [OrangeJuice, CheeseCroissant]          |   2079  |
|              [CherryTart, OperaCake]              |   2067  |
|             [ApricotDanish, OperaCake]            |   2046  |
|         [ApricotCroissant, BlueberryTart]         |   2042  |
|           [StrawberryCake, NapoleonCake]          |   2032  |
|              [CoffeeEclair, ApplePie]             |   1816  |
|            [CoffeeEclair, AlmondTwist]            |   1788  |
|              [ApplePie, AlmondTwist]              |   1785  |
|             [BerryTart, BottledWater] 

Next, we apply the **`extract_features()`** method of this newly trained **`top100_freq_patterns`** model on the **`test`** data set.

In [20]:
features = top100_freq_patterns.extract_features(train)

Preprocessing complete. Found 73566 unique transactions.

Once the features are extracted, we can use them downstream in other applications such as **clustering**, **classification**, **churn prediction**, **recommender systems** etc.

In [23]:
features.print_rows(num_rows=10, max_row_width=120, max_column_width=100)

+---------+----------+
| Receipt | StoreNum |
+---------+----------+
|   8829  |    7     |
|  35020  |    14    |
|  38810  |    20    |
|   6015  |    7     |
|  68521  |    3     |
|   6447  |    5     |
|  12900  |    10    |
|  49658  |    20    |
|  20227  |    19    |
|  47850  |    2     |
+---------+----------+
+-----------------------------------------------------------------------------------------------------+
|                                          extracted_features                                         |
+-----------------------------------------------------------------------------------------------------+
| [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ... |
| [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ... |
| [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ... |
| [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

### Example: Employee Space Clustering by using occurrences of frequency patterns

First we provide an aggregated form of our data by selecting one **Selling Employee (`EmpId`)** at random.

In [24]:
emps = train.groupby(['Receipt', 'StoreNum'],
                     {'EmpId': agg.SELECT_ONE('EmpId')})
emps

Receipt,StoreNum,EmpId
8829,7,7
35020,14,14
38810,20,20
6015,7,7
68521,3,3
6447,5,5
12900,10,35
49658,20,20
20227,19,50
47850,2,23


Next, we count the instances that each of the **`top100_freq_patterns`** occurs per **`EmpId`**.

In [25]:
emp_space = emps.join(features).groupby('EmpId', {'all_features': agg.SUM('extracted_features')})
emp_space

EmpId,all_features
49,"[41.0, 53.0, 35.0, 42.0,45.0, 30.0, 30.0, 31.0, ..."
13,"[126.0, 130.0, 95.0,77.0, 91.0, 100.0, 10 ..."
14,"[140.0, 113.0, 107.0,101.0, 116.0, 121.0, ..."
48,"[28.0, 34.0, 28.0, 23.0,29.0, 23.0, 19.0, 23.0, ..."
20,"[133.0, 129.0, 123.0,110.0, 109.0, 108.0, ..."
12,"[15.0, 19.0, 16.0, 14.0,16.0, 11.0, 14.0, 13.0, ..."
22,"[52.0, 48.0, 37.0, 45.0,46.0, 41.0, 35.0, 36.0, ..."
36,"[35.0, 35.0, 46.0, 38.0,33.0, 30.0, 22.0, 32.0, ..."
10,"[38.0, 40.0, 35.0, 40.0,38.0, 36.0, 36.0, 29.0, ..."
29,"[17.0, 10.0, 13.0, 19.0,16.0, 13.0, 16.0, 19.0, ..."


Finally, we train a **kmeans** algorithm to produce a **3 centered cluster**.

In [26]:
cl_model = gl.kmeans.create(emp_space, 
                            features = ['all_features'], 
                            num_clusters=3)

Batch size is larger than the input dataset. Switching to an exact Kmeans method.

Choosing initial cluster centers with Kmeans++.

+---------------+-----------+

| Center number | Row index |

+---------------+-----------+

...

+-----------+-------------------------------+

| Iteration | Number of changed assignments |

+-----------+-------------------------------+

| 1         | 0                             |

+-----------+-------------------------------+

In [27]:
emp_space['cluster_id'] = cl_model['cluster_id']['cluster_id']
emp_space

EmpId,all_features,cluster_id
49,"[41.0, 53.0, 35.0, 42.0,45.0, 30.0, 30.0, 31.0, ...",1
13,"[126.0, 130.0, 95.0,77.0, 91.0, 100.0, 10 ...",0
14,"[140.0, 113.0, 107.0,101.0, 116.0, 121.0, ...",2
48,"[28.0, 34.0, 28.0, 23.0,29.0, 23.0, 19.0, 23.0, ...",1
20,"[133.0, 129.0, 123.0,110.0, 109.0, 108.0, ...",2
12,"[15.0, 19.0, 16.0, 14.0,16.0, 11.0, 14.0, 13.0, ...",1
22,"[52.0, 48.0, 37.0, 45.0,46.0, 41.0, 35.0, 36.0, ...",1
36,"[35.0, 35.0, 46.0, 38.0,33.0, 30.0, 22.0, 32.0, ...",1
10,"[38.0, 40.0, 35.0, 40.0,38.0, 36.0, 36.0, 29.0, ...",1
29,"[17.0, 10.0, 13.0, 19.0,16.0, 13.0, 16.0, 19.0, ...",1


In [29]:
gl.canvas.set_target('ipynb')
emp_space.show(x='cluster_id', y = 'StoreNum', view='Bar Chart')